## Setup

In [112]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
import scipy.io
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LogisticRegression
from pyrcn.echo_state_network import ESNClassifier
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import cupy as cp
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir
from reservoirpy.nodes import Ridge
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import itertools

## Parameters

In [113]:
subject_id = 1

## Load data

In [114]:
npz_filename_T = f'./data/train/A0{subject_id}T.npz'
npz_filename_E = f'./data/eval/A0{subject_id}E.npz'

data_T = np.load(npz_filename_T)
data_E = np.load(npz_filename_E)

In [115]:
X_train = data_T['X']

# y_train = np.asarray([np.asarray([item-6 for i in range(X_train.shape[1])]) for item in data_T['y']])
y_train = data_T['y'] - 6

X_eval = data_E['X']
y_true = data_E['y_true']

In [116]:
X_train.shape, y_train.shape, np.max(X_train), np.min(X_train), np.mean(X_train), np.std(X_train)

((288, 22, 751),
 (288,),
 np.float64(4.216004148744056e-05),
 np.float64(-4.2082318582766146e-05),
 np.float64(-2.6516604026355054e-09),
 np.float64(5.388162290701335e-06))

## One-Hot encode labels

In [117]:
lb = LabelBinarizer()

y_train_onehot = lb.fit_transform(y_train)
y_true_onehot = lb.transform(y_true)

## CSP Transformation

In [118]:
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

csp = csp.fit(X_train, y_train)

Computing rank from data with rank=None
    Using tolerance 5e-05 (2.2e-16 eps * 22 dim * 1e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.


In [119]:
csp.filters_.shape, X_train[0].shape

((22, 22), (22, 751))

In [120]:
[sample.T.shape for sample in X_train][0]

(751, 22)

In [121]:
X_train_csp = np.asarray([csp.transform(sample.T) for sample in X_train])
X_eval_csp = np.asarray([csp.transform(sample.T) for sample in X_eval])

In [122]:
X_train.shape, X_train_csp.shape, X_eval_csp.shape

((288, 22, 751), (288, 751, 4), (288, 751, 4))

In [ ]:
X_train_csp.shape, y_train.shape, np.max(X_train_csp), np.min(X_train_csp), np.mean(X_train_csp), np.std(X_train_csp)

((288, 751, 4),
 (288,),
 np.float64(3.9311456914301317),
 np.float64(-25.50502669610828),
 np.float64(-1.5510004655988325),
 np.float64(2.3518185063966763))

## Train

In [124]:
# def extract_states(X_data, reservoir, aggregate='mean'):
#     states_list = []

#     pbar = tqdm(range(len(X_data)))

#     for i in pbar:
#         data = X_data[i].T

#         states = []
#         for j in range(data.shape[0]):
#             input = data[j].reshape(data[j].shape[0], 1)

#             state = reservoir.run(input, workers=-1)

#             states.append(state)

#         if aggregate == 'final':
#             feature = states[-1:, :]
#         elif aggregate == 'mean':
#             feature = np.mean(states, axis=0, keepdims=True)
#         elif aggregate == 'concat':
#             # Combine multiple statistics
#             feature = np.concatenate([
#                 states[-1:, :],
#                 np.mean(states, axis=0, keepdims=True),
#                 np.std(states, axis=0, keepdims=True)
#             ], axis=1)
        
#         states_list.append(feature)
    
#     return np.vstack(states_list)

def extract_states(X_data, reservoir, aggregate='mean'):
    states_list = []

    for i in range(len(X_data)):
        data = X_data[i]

        states = reservoir.run(data, workers=-1)

        if aggregate == 'final':
            feature = states[-1:, :]
        elif aggregate == 'mean':
            feature = np.mean(states, axis=0, keepdims=True)
        elif aggregate == 'concat':
            # Combine multiple statistics
            feature = np.concatenate([
                states[-1:, :],
                np.mean(states, axis=0, keepdims=True),
                np.std(states, axis=0, keepdims=True)
            ], axis=1)
        
        states_list.append(feature)
    
    return np.vstack(states_list)

In [ ]:
rpy.set_seed(42)

# Expected format: (time_steps, channels)
reservoir = Reservoir(500, lr=0.7, sr=1.5)

readout = Ridge(ridge=1e3)

aggregate = 'mean'

X_train_csp_states = extract_states(X_train_csp, reservoir, aggregate=aggregate)
X_eval_csp_states = extract_states(X_eval_csp, reservoir, aggregate=aggregate)

readout.fit(X_train_csp_states, y_train_onehot)

y_train_csp_pred = readout.run(X_train_csp_states)
y_eval_csp_pred = readout.run(X_eval_csp_states)

y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred_class):.4f}")
print(f'Train Kappa: {cohen_kappa_score(y_train, y_train_pred_class)}')
print(f"Eval Accuracy: {accuracy_score(y_true, y_eval_pred_class):.4f}")
print(f'Eval Kappa: {cohen_kappa_score(y_true, y_eval_pred_class)}')

print("\nClassification Report:")
print(classification_report(y_true, y_eval_pred_class))

Train Accuracy: 0.6354
Train Kappa: 0.5138888888888888
Eval Accuracy: 0.6146
Eval Kappa: 0.48611111111111116

Classification Report:
              precision    recall  f1-score   support

           1       0.67      0.83      0.75        72
           2       0.92      0.64      0.75        72
           3       0.75      0.04      0.08        72
           4       0.47      0.94      0.63        72

    accuracy                           0.61       288
   macro avg       0.70      0.61      0.55       288
weighted avg       0.70      0.61      0.55       288



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# PCA visualization of reservoir states in 3D
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_train_csp_states)

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {np.sum(pca.explained_variance_ratio_):.3f}")

# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Define colors for each class
colors = {1: 'red', 2: 'blue', 3: 'green', 4: 'orange'}
label_names = {1: 'Left Hand', 2: 'Right Hand', 3: 'Feet', 4: 'Tongue'}

# Plot each class
for label in np.unique(y_train):
    mask = y_train == label
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2], 
               c=colors[label], label=label_names[label], alpha=0.7, s=50)

ax.set_title('3D PCA of Reservoir States')
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.2%} variance)')
ax.legend()

plt.tight_layout()
plt.show()

In [95]:
X_train_csp_states.shape, y_train_onehot.shape

((288, 4, 500), (288, 4))

In [137]:
window_samples = 100  # 2 seconds @ 250 Hz
step_samples = 10     # 40ms step, as per winner's PDF [cite: 145]

all_trial_predictions = []

washout_time = 125

sample_step = 10

# Loop over each trial
for trial_data in X_eval_csp:
    states = []
    trial_predictions = []

    for i in range(0, trial_data.shape[0], sample_step):
        input = trial_data[i].reshape(1, trial_data[i].shape[0])
        
        state = reservoir.run(input)

        if i >= washout_time:
            states.append(state)

            mean_state = np.mean(states, axis=0, keepdims=True)

            y_pred = readout.run(X_train_csp_states)
            y_pred_class = np.argmax(y_pred, axis=1) + 1

            trial_predictions.append(y_pred_class)

    all_trial_predictions.append(trial_predictions)

    # # Slide the 2-second window
    # state = np.zeros()
    # for start in range(0, trial_data.shape[2] - window_samples, step_samples):
    #     end = start + window_samples
    #     window_data = trial_data[:, start:end]

    #     # Reshape for the pipeline: (1, n_channels, n_samples)
    #     window_data_reshaped = window_data.reshape(1, *window_data.shape)
        
    #     # Make a prediction using our trained causal model
    #     prediction = clf.predict(window_data_reshaped)
    #     trial_predictions.append(prediction[0])

all_trial_predictions.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Transpose the prediction matrix so we can score each time point
predictions_by_time = np.array(all_trial_predictions).T

kappa_scores = []
for y_pred_at_time_t in predictions_by_time:
    # Map MNE labels (769...) to simple labels (1...)
    y_pred_mapped = np.array([pred for pred in y_pred_at_time_t])
    
    # Calculate kappa for this single time slice
    kappa = cohen_kappa_score(y_true, y_pred_mapped)
    kappa_scores.append(kappa)
    acc = accuracy_score(y_true, y_pred_mapped)

# --- 5. Find the Final Score ---
max_kappa = np.max(kappa_scores)
max_acc = np.max(acc)

print(f"Time course of Kappa calculated.")
print(f"Maximum Kappa value: {max_kappa:.3f}")
print(f"Maximum Accuracy value: {max_acc:.3f}")

## Fine Tune

In [ ]:
def build_model(rs, lr, sr, alpha):
    reservoir = Reservoir(rs, lr=lr, sr=sr)
    readout = Ridge(ridge=alpha)

    return reservoir, readout

In [ ]:
rs_ls = [500, 750]
alpha_ls = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
lr_ls = [0.3, 0.5, 0.7]
sr_ls = [0.5, 0.7, 0.9, 1.2, 1.5]
aggregate_ls = ['final', 'mean', 'concat']

rpy.set_seed(42)

results_list = []

model_params = list(itertools.product(rs_ls, lr_ls, sr_ls, alpha_ls, aggregate_ls))
total_iterations = len(model_params)

pbar = tqdm(total=total_iterations, desc="Grid Search Progress")

for (rs, lr, sr, alpha, aggregate) in model_params:
    params = {'rs': rs, 'lr': lr, 'sr': sr, 'alpha': alpha}
    pbar.set_postfix({'rs': rs, 'lr': lr, 'sr': sr, 'alpha': alpha})
    # print('-')

    reservoir, readout = build_model(rs=rs, lr=lr, sr=sr, alpha=alpha)

    X_train_csp_states = extract_states(X_train_csp, reservoir, aggregate=aggregate)
    X_eval_csp_states = extract_states(X_eval_csp, reservoir, aggregate=aggregate)

    readout.fit(X_train_csp_states, y_train_onehot)

    y_train_csp_pred = readout.run(X_train_csp_states)
    y_eval_csp_pred = readout.run(X_eval_csp_states)

    y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
    y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

    acc_train = accuracy_score(y_train, y_train_pred_class)
    kappa_train = cohen_kappa_score(y_train, y_train_pred_class)

    acc_eval = accuracy_score(y_true, y_eval_pred_class)
    kappa_eval = cohen_kappa_score(y_true, y_eval_pred_class)

    f1_score_eval = f1_score(y_true, y_eval_pred_class, average='weighted')

    current_result = {
        'rs': rs,
        'lr': lr,
        'sr': sr,
        'alpha': alpha,
        'aggregate': aggregate,
        'acc_train': acc_train,
        'kappa_train': kappa_train,
        'acc_eval': acc_eval,
        'kappa_eval': kappa_eval,
        'f1_weighted': f1_score_eval
    }

    print(f'{kappa_eval}')

    results_list.append(current_result)

    pbar.update(1)

print("Parameter search complete.")
                    
df_results = pd.DataFrame(results_list)

# df_results.to_csv('tuning_results.csv', index=False)

In [ ]:
df_sorted = df_results.sort_values(by='kappa_eval', ascending=False)

print("Top 5 Results:")
print(df_sorted.head(5))

best_params = df_sorted.iloc[0].to_dict()

print("\nBest Parameter Set:")
print(best_params)

## Per subject

In [ ]:
subject_id_ls = [i for i in range(1, 10)]

rpy.set_seed(42)

reservoir = Reservoir(1000, lr=0.5, sr=1.5)

readout = Ridge(ridge=1e2)

for subject_id in tqdm(subject_id_ls):
    # Load data
    npz_filename_T = f'./data/train/A0{subject_id}T.npz'
    npz_filename_E = f'./data/eval/A0{subject_id}E.npz'

    data_T = np.load(npz_filename_T)
    data_E = np.load(npz_filename_E)

    # 
    X_train = data_T['X']

    # y_train = np.asarray([np.asarray([item-6 for i in range(X_train.shape[1])]) for item in data_T['y']])
    y_train = data_T['y'] - 6

    X_eval = data_E['X']
    y_true = data_E['y_true']

    # 
    X_train = data_T['X']

    y_train = data_T['y'] - 6

    X_eval = data_E['X']
    y_true = data_E['y_true']

    lb = LabelBinarizer()

    y_train_onehot = lb.fit_transform(y_train)
    y_true_onehot = lb.transform(y_true)

    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

    csp = csp.fit(X_train, y_train)

    X_train_csp = np.asarray([csp.transform(sample.T) for sample in X_train]).transpose(0, 2, 1)
    X_eval_csp = np.asarray([csp.transform(sample.T) for sample in X_eval]).transpose(0, 2, 1)

    aggregate = 'concat'

    X_train_csp_states = extract_states(X_train_csp, reservoir, aggregate=aggregate)
    X_eval_csp_states = extract_states(X_eval_csp, reservoir, aggregate=aggregate)

    readout.fit(X_train_csp_states, y_train_onehot)

    y_train_csp_pred = readout.run(X_train_csp_states)
    y_eval_csp_pred = readout.run(X_eval_csp_states)

    y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
    y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

    print(25*'==')
    print(f'SUBJECT ID: {subject_id}')
    print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred_class):.4f}")
    print(f'Train Kappa: {cohen_kappa_score(y_train, y_train_pred_class)}')
    print(f"Eval Accuracy: {accuracy_score(y_true, y_eval_pred_class):.4f}")
    print(f'Eval Kappa: {cohen_kappa_score(y_true, y_eval_pred_class)}')

    print("\nClassification Report:")
    print(classification_report(y_true, y_eval_pred_class))
    
